In [2]:
import random
import numpy as np
import time
import math
import uuid
import csv
import os
from math import cos, exp, sqrt, pi

In [3]:
POP_SIZE     = 100
N_EVALS      = 100000
N_REPLICATES = 30
MIN_LEN      = 3
MAX_LEN      = 100
gene_low, gene_high = -2.0, 2.0
SIGMA_INIT    = 0.5
P_DIM_CHANGE  = 0.01

target_map = {
    'Sphere':    30,
    'Rastrigin': 30,
    'Griewank':  30,
    'Ackley':    30,
    'Koza':      10,
    'Knapsack':  20,
    'Parity':    64,
    'SantaFe':   100
}

time_str = time.strftime('%Y%m%d_%H%M%S')
out_dir   = f'CMA_ES_results_{time_str}'
os.makedirs(out_dir, exist_ok=True)
csv_summary = os.path.join(out_dir, 'CMA_ES_results.csv')
csv_history = os.path.join(out_dir, 'CMA_ES_history.csv')

In [4]:
def mkPenalizer(d_target, lam=10, deadzone=5):
    def _pen(ind):
        dev = abs(len(ind) - d_target)
        return 0 if dev <= deadzone else lam * dev
    return _pen

In [5]:
def sphere(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    return lambda ind: sum(x*x for x in ind) + pen(ind)

def rastrigin(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    return lambda ind: 10*len(ind) + sum(x*x - 10*cos(2*pi*x) for x in ind) + pen(ind)

def griewank(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        s = sum(x*x for x in ind)
        p = np.prod([cos(x/sqrt(i+1)) for i,x in enumerate(ind)])
        return 1 + s/4000.0 - p + pen(ind)
    return f

def ackley(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        d = len(ind)
        s = sum(x*x for x in ind)
        c = sum(cos(2*pi*x) for x in ind)
        return -20*exp(-0.2*sqrt(s/d)) - exp(c/d) + 20 + math.e + pen(ind)
    return f

def koza(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    xs = np.linspace(-1,1,20)
    def f(ind):
        err = 0.0
        for x in xs:
            t = x**4 + x**3 + x**2 + x
            p = sum(w*(x**i) for i,w in enumerate(ind))
            err += (p - t)**2
        return err + pen(ind)
    return f

def knapsack(d_target, lam, items, capacity):
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        sel = [int(round(x))%2 for x in ind[:len(items)]]
        w_sum = sum(w for (w,v),s in zip(items, sel) if s)
        v_sum = sum(v for (w,v),s in zip(items, sel) if s)
        over = max(0, w_sum - capacity)
        return -v_sum + 1000*over + pen(ind)
    return f

def parity(N, lam):
    d_target = 2**N
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        table = [int(round(x))%2 for x in ind[:d_target]]
        errs = 0
        for i in range(d_target):
            target = bin(i).count('1')%2
            errs += 0 if (i < len(table) and table[i]==target) else 1
        return errs + pen(ind)
    return f

def santaFe(d_target, lam):
    world=[]; sx=sy=0
    with open('santafe_trail.txt') as f:
        for y,line in enumerate(f):
            for x,c in enumerate(line.rstrip('\n')):
                if c=='#': world.append(1)
                elif c in ('.','S'):
                    world.append(0)
                    if c=='S': sx,sy = x,y
    W = len(open('santafe_trail.txt').readline().rstrip('\n'))
    H = len(world)//W
    pen = mkPenalizer(d_target, lam)
    def f(ind, max_steps=400):
        dir_ = 0; x,y = sx,sy; food=0
        visited = world.copy()
        idx = lambda xx,yy: (yy%H)*W+(xx%W)
        for i in range(min(len(ind), max_steps)):
            if visited[idx(x,y)]==1:
                food +=1; visited[idx(x,y)]=0
            act = int(abs(ind[i]))%3
            if act==0:
                dx,dy=[(0,1),(1,0),(0,-1),(-1,0)][dir_]
                x+=dx; y+=dy
            elif act==1:
                dir_=(dir_+1)%4
            else:
                dir_=(dir_-1)%4
        return -food + pen(ind)
    return f

In [6]:
def computeHyperparams(n_dim, mu):
    w = np.log(mu+0.5) - np.log(np.arange(1,mu+1))
    w /= np.sum(w)
    mu_eff = 1.0/np.sum(w**2)
    c_c  = (4+mu_eff/n_dim)/(n_dim+4+2*mu_eff/n_dim)
    c_s  = (mu_eff+2)/(n_dim+mu_eff+5)
    c1   = 2/((n_dim+1.3)**2 + mu_eff)
    c_mu = min(1-c1, 2*(mu_eff-2+1/mu_eff)/((n_dim+2)**2+mu_eff))
    d_s  = 1 + 2*max(0, sqrt((mu_eff-1)/(n_dim+1))-1) + c_s
    chi_n= sqrt(n_dim)*(1 - 1/(4*n_dim) + 1/(21*n_dim**2))
    return w, mu_eff, c_c, c_s, c1, c_mu, d_s, chi_n

In [7]:
def runCMAES(fit, name):
    n     = target_map[name]
    m     = np.random.uniform(gene_low, gene_high, size=n)
    C     = np.eye(n)
    sigma = SIGMA_INIT
    p_c   = np.zeros(n)
    p_s   = np.zeros(n)
    evals = 0
    best_hist = []
    mu = POP_SIZE // 2
    w, mu_eff, c_c, c_s, c1, c_mu, d_s, chi_n = computeHyperparams(n, mu)

    while evals < N_EVALS:
        if random.random() < P_DIM_CHANGE:
            if random.random() < 0.5 and len(m) < MAX_LEN:
                m = np.append(m, random.uniform(gene_low, gene_high))
                C = np.pad(C, ((0,1),(0,1)), constant_values=0); C[-1,-1]=1.0
                p_c = np.append(p_c, 0.0); p_s = np.append(p_s, 0.0)
            elif len(m) > MIN_LEN:
                m = m[:-1]; C = C[:-1,:-1]; p_c = p_c[:-1]; p_s = p_s[:-1]
            n = len(m)
            w, mu_eff, c_c, c_s, c1, c_mu, d_s, chi_n = computeHyperparams(n, mu)

        D2, B = np.linalg.eigh(C)
        D    = np.sqrt(np.abs(D2))
        arz  = np.random.randn(POP_SIZE, n)
        arx  = m + sigma * (arz @ (B * D).T)
        fits = [fit(x.tolist()) for x in arx]
        evals += POP_SIZE

        idx = np.argsort(fits)
        arx, arz = arx[idx], arz[idx]
        fits = [fits[i] for i in idx]
        best = fits[0]
        best_hist.append(best)

        if evals % (POP_SIZE * 10) == 0:
            print(f'{name} Evals {evals}: Best = {best:.6f}, Len = {len(m)}')

        old_m = m.copy()
        m = np.dot(w, arx[:mu])
        y = (m - old_m) / sigma
        inv_sqrtC = B @ np.diag(1.0/D) @ B.T
        p_s = (1-c_s)*p_s + sqrt(c_s*(2-c_s)*mu_eff) * (inv_sqrtC @ y)
        norm_ps = np.linalg.norm(p_s)
        h_sigma = int(norm_ps < (1.4+2/(n+1)) * sqrt(1 - (1-c_s)**(2*evals/POP_SIZE)))
        p_c = (1-c_c)*p_c + h_sigma*sqrt(c_c*(2-c_c)*mu_eff) * y
        artmp = (arz[:mu].T * w).T
        C = ((1-c1-c_mu)*C
             + c1*(np.outer(p_c, p_c) + (1-h_sigma)*c_c*(2-c_c)*C)
             + c_mu*(artmp.T @ artmp))
        C = (C + C.T)/2
        diag = np.clip(np.diag(C), 1e-8, 1e8)
        C[np.diag_indices(len(C))] = diag
        expnt = (c_s/d_s)*(norm_ps/chi_n - 1)
        expnt = np.clip(expnt, -10, 10)
        sigma *= math.exp(expnt)
        sigma = float(np.clip(sigma, 1e-8, 1e+1))

    return m, best_hist

In [8]:
with open(csv_summary, 'w', newline='') as fs, open(csv_history, 'w', newline='') as fh:
    writer_sum = csv.DictWriter(fs, fieldnames=['run_id','benchmark','replicate','seed','best','len_final','cpu_s'])
    writer_hist = csv.DictWriter(fh, fieldnames=['run_id','benchmark','replicate','seed','evals','fitness'])
    writer_sum.writeheader()
    writer_hist.writeheader()

    tmp_items = [(random.randint(1,20), random.randint(10,100)) for _ in range(20)]
    benchmarks = {
        'Sphere':    sphere(30,10),
        'Rastrigin': rastrigin(30,10),
        'Griewank':  griewank(30,10),
        'Ackley':    ackley(30,10),
        'Koza':      koza(10,10),
        'Knapsack':  knapsack(20,0,tmp_items,50),
        'Parity':    parity(6,10),
        'SantaFe':   santaFe(100,10),
    }

    for name, func in benchmarks.items():
        for rep in range(N_REPLICATES):
            seed = random.randint(0,2**31-1)
            random.seed(seed); np.random.seed(seed)
            start = time.perf_counter()
            best_vec, history = runCMAES(func, name)
            cpu = time.perf_counter() - start
            best_val = round(func(best_vec.tolist()), 6)
            run_id = uuid.uuid4()
            writer_sum.writerow({
                'run_id': run_id,
                'benchmark': name,
                'replicate': rep,
                'seed': seed,
                'best': best_val,
                'len_final': len(best_vec),
                'cpu_s': round(cpu,4)
            })
            for i, fitval in enumerate(history):
                ev = (i+1) * POP_SIZE
                writer_hist.writerow({
                    'run_id': run_id,
                    'benchmark': name,
                    'replicate': rep,
                    'seed': seed,
                    'evals': ev,
                    'fitness': fitval
                })
print(f'Results written to {csv_summary} and history to {csv_history}')

Sphere Evals 1000: Best = 42.765252, Len = 30
Sphere Evals 2000: Best = 5.034142, Len = 30
Sphere Evals 3000: Best = 0.932350, Len = 30
Sphere Evals 4000: Best = 0.150378, Len = 30
Sphere Evals 5000: Best = 0.034916, Len = 30
Sphere Evals 6000: Best = 0.009177, Len = 30
Sphere Evals 7000: Best = 0.002063, Len = 30
Sphere Evals 8000: Best = 0.000479, Len = 29
Sphere Evals 9000: Best = 0.000162, Len = 29
Sphere Evals 10000: Best = 0.000026, Len = 29
Sphere Evals 11000: Best = 0.000007, Len = 29
Sphere Evals 12000: Best = 0.000002, Len = 29
Sphere Evals 13000: Best = 0.000001, Len = 29
Sphere Evals 14000: Best = 0.000000, Len = 29
Sphere Evals 15000: Best = 1.848064, Len = 30
Sphere Evals 16000: Best = 0.368842, Len = 30
Sphere Evals 17000: Best = 0.165593, Len = 30
Sphere Evals 18000: Best = 0.041029, Len = 30
Sphere Evals 19000: Best = 0.006598, Len = 30
Sphere Evals 20000: Best = 0.002201, Len = 30
Sphere Evals 21000: Best = 0.082067, Len = 31
Sphere Evals 22000: Best = 0.027545, Len =